In [511]:
import imaplib
import getpass
import email

from email.header import decode_header, make_header

## Подключениек серверу IMAP

In [512]:
imap = imaplib.IMAP4_SSL('imap.gmail.com')
imap.login('widosmile@gmail.com', getpass.getpass())

········


('OK', [b'widosmile@gmail.com authenticated (Success)'])

## Список доступных папок
Junk - это спам
Проблема - название ящика для подключения имеет уникальное значение для каждого пользователя

In [513]:
imap_list = imap.list(pattern='*')

In [514]:
imap_list

('OK',
 [b'(\\HasNoChildren) "/" "INBOX"',
  b'(\\HasNoChildren) "/" "Notes"',
  b'(\\HasChildren \\Noselect) "/" "[Gmail]"',
  b'(\\HasNoChildren \\Important) "/" "[Gmail]/&BBIEMAQ2BD0EPgQ1-"',
  b'(\\All \\HasNoChildren) "/" "[Gmail]/&BBIEQQRP- &BD8EPgRHBEIEMA-"',
  b'(\\HasNoChildren \\Trash) "/" "[Gmail]/&BBoEPgRABDcEOAQ9BDA-"',
  b'(\\HasNoChildren \\Sent) "/" "[Gmail]/&BB4EQgQ,BEAEMAQyBDsENQQ9BD0ESwQ1-"',
  b'(\\Flagged \\HasNoChildren) "/" "[Gmail]/&BB8EPgQ8BDUERwQ1BD0EPQRLBDU-"',
  b'(\\HasNoChildren \\Junk) "/" "[Gmail]/&BCEEPwQwBDw-"',
  b'(\\Drafts \\HasNoChildren) "/" "[Gmail]/&BCcENQRABD0EPgQyBDgEOgQ4-"'])

### Встроеные методы не позволяют выбирать по началу заголовка, пришлось написать функцию самому
Однако, судя по информации и примерах кода в интернете, в некоторых случаях можно по общему имени выбирать (у меня не вышло)

In [515]:
def get_foldel_name(imap_list, general_name):
    result = []
    
    for i in imap_list[1]:
        l = i.decode().split(' "/" ')
        if general_name in l[0]:
            result.append((l[0], l[1]))
            
    return tuple(result)

# Spam

In [516]:
spam_folder = get_foldel_name(imap_list, 'Junk')
spam_folder

(('(\\HasNoChildren \\Junk)', '"[Gmail]/&BCEEPwQwBDw-"'),)

In [517]:
imap.select(spam_folder[0][1])

('OK', [b'7'])

In [518]:
ids = imap.search(None, 'ALL')
ids

('OK', [b'1 2 3 4 5 6 7'])

In [519]:
post_ids = ids[1][0].decode().split(' ')
post_ids = [p_id.encode() for p_id in post_ids]
post_ids

[b'1', b'2', b'3', b'4', b'5', b'6', b'7']

In [520]:
status, letter = imap.fetch(post_ids[0], '(RFC822)')
status, letter

('OK',
 [(b'1 (RFC822 {125010}',
   b'Delivered-To: widosmile@gmail.com\r\nReceived: by 2002:adf:8062:0:0:0:0:0 with SMTP id 89csp4360267wrk;\r\n        Tue, 22 Sep 2020 04:19:31 -0700 (PDT)\r\nX-Google-Smtp-Source: ABdhPJxXok+Qh1xvduEjoWr3uZGIp9KADAPBydMpHrTR+Kg7UXRNx88Q9kJMIfjDiV7DR9CoyoRP\r\nX-Received: by 2002:a7b:c14d:: with SMTP id z13mr448873wmi.107.1600773571692;\r\n        Tue, 22 Sep 2020 04:19:31 -0700 (PDT)\r\nARC-Seal: i=1; a=rsa-sha256; t=1600773571; cv=none;\r\n        d=google.com; s=arc-20160816;\r\n        b=BE5DNDfTwrQizRrNiEXTgQMzKQp/2O7e6JaKMb0lqm+QEA53TpXRYiC1KAmIVkH6uH\r\n         w1hCCUnfTyxB4EC4q31cdjUvRbwQ+kZug4SB00HmuGHAUWKI6oApERJSa545ocG5Pz45\r\n         inxJbp06xatUxY26rSbYbFMeE+V1NEAbVV/e/MlrKTg92MmfQ9XYzYLuQw1SFDFZPlTA\r\n         TF5b+LwbHRI27V2IF5hoow6HQ/ygFjQeTvQnuzYeKs4z0iT0RiXWRcHSxqIS31pQBaNv\r\n         WLAtPEEukGY/mGy2YH/pRBdtYBZI99adWr3V0lgppzCVPaTbBXgFW4221MgRa54a/yiP\r\n         1MjA==\r\nARC-Message-Signature: i=1; a=rsa-sha256; c=relaxed/rel

## Letter parsing
Используем модуль email

In [521]:
msg = email.message_from_bytes(letter[0][1], _class = email.message.EmailMessage)
msg

In [522]:
msg['Subject']

'=?utf-8?b?4pqh77iPINCQ0L3QvtC90YHRiyDQvdC10LTQtdC70LgsINC60L7RgtC+0YDRi9C1INCy0Ysg0L3QtSDQt9Cw0YXQvtGC0LjRgtC1INC/0YDQvtC/0YPRgdGC0LjRgtGM?='

In [398]:
parsed_letter = dict(msg)
print('keys: ', parsed_letter.keys())
#parsed_letter

keys:  dict_keys(['Delivered-To', 'Received', 'X-Google-Smtp-Source', 'X-Received', 'ARC-Seal', 'ARC-Message-Signature', 'ARC-Authentication-Results', 'Return-Path', 'Received-SPF', 'Authentication-Results', 'Content-Type', 'MIME-Version', 'From', 'Precedence', 'Feedback-ID', 'List-ID', 'Subject', 'Date', 'Sender', 'X-Authenticated-User', 'Errors-To', 'X-Complaints-To', 'To', 'List-Unsubscribe', 'Message-Id', 'DKIM-Signature'])


### Пример оновных значений
Поля в imap закодированы в MIME формате  
Использую decode_header  
__!__ В return[0][1] возвращает изначальную кодировку

In [399]:
for key in parsed_letter.keys():
    
    parsed_letter[key] = str(make_header(decode_header(parsed_letter[key])))

In [497]:
print('From: ', parsed_letter['From'])
#print('Sender: ', parsed_letter['Sender'])
print('Subject: ', parsed_letter['Subject']) 
print('Date: ', parsed_letter['Date']) 
print('Content-Type: ', parsed_letter['Content-Type']) 

From:  Роман Павлов | SF Education <education@sf.education>
Subject:  ⚡️ Анонсы недели, которые вы не захотите пропустить
Date:  Tue, 22 Sep 2020 11:10:35 +0000 (GMT)
Content-Type:  multipart/alternative;
 boundary="===============5354314217074122774=="


In [401]:
msg.get_payload()

### Оставшиеся закодированными байты никакой информации не несут (все содержимое письма здесь присутствует)

### Однако тело может быть закодировано уже в другой кодировке (Отличие '=?utf-8?Q?' и '=?utf-8?b?' в начале)
### Кроме этого тело может состоять как из одной части, так и из нескольких (в случае нескольких нужно дополнительно найти нужное и декодировать его. Части могут как состоять из мусорных элементов, наподобие ?,!,<, так и пердставлять собой текст письма в разном формате)
Чтобы бороться с данной проблемой, написана функция read_body()  
Ниже показан пример письма из нескольких частей

In [488]:
c = msg.get_body(preferencelist=('plain', 'html'))

In [489]:
msg.get_payload()[0].get_payload(decode=True).decode('utf-8','ignore')

''

In [490]:
msg.get_payload()[1].get_payload(decode=True).decode('utf-8','ignore')

'new Дайджест >>\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u

In [491]:
msg.get_payload()[2].get_payload(decode=True).decode('utf-8','ignore')

'<!DOCTYPE html>\r\n<html xmlns:o="urn:schemas-microsoft-com:office:office" style="width:100%;font-family:tahoma, verdana, segoe, sans-serif;-webkit-text-size-adjust:100%;-ms-text-size-adjust:100%;padding:0;Margin:0">\r\n<head>\r\n<meta charset="UTF-8">\r\n<meta content="width=device-width, initial-scale=1" name="viewport">\r\n<meta name="x-apple-disable-message-reformatting">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<meta content="telephone=no" name="format-detection">\r\n<title>new Дайджест</title>\r\n<!--[if (mso 16)]> <style type="text/css"> a {text-decoration: none;} </style> <![endif]--><!--[if gte mso 9]><style>sup { font-size: 100% !important; }</style><![endif]--><!--[if gte mso 9]> <xml> <o:OfficeDocumentSettings> <o:AllowPNG></o:AllowPNG> <o:PixelsPerInch>96</o:PixelsPerInch> </o:OfficeDocumentSettings> </xml> <![endif]-->\r\n\r\n<style type="text/css">\r\n .rollover:hover .rollover-first { max-height:0px!important; display:none!important; } .rollover:hove

In [492]:
def read_body(msg):
    # Необходимо, чтобы функция не принимала заглушки наподобие b' ' за текст
    # Это не мешает обойти систему послав очень длинную заглушку. Но это маловероятно
    body_len = 0
    
    if msg.is_multipart():
        for part in msg.walk():
            ctype = part.get_content_type()
            cdispo = str(part.get('Content-Disposition'))

            # Пропустить любые text/plain (txt) вложения
            if ctype == 'text/plain' and 'attachment' not in cdispo:
                if body_len < len(part.get_payload(decode=True)):
                    body = part.get_payload(decode=True)  # decode
                    body_len = len(body)
        # Письмо без деления на части. plain text без вложений. !Не дает гарантии, что это будет именно текст
        # может быть только вложение
    else:
        body = msg.get_payload(decode=True)
        
    return body.decode('utf-8','ignore')

In [501]:
body = read_body(msg)

In [502]:
body

'new Дайджест >>\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u2005\xa0\u200c\u

### Не понятно, что делать с мусорными символами в начале. Возможно удалить фильтрацией. Но пока не нашел способа автоматического и без недостатков

! Данную работу должен осуществлять модуль предобработки данных

In [495]:
list(set(list(read_body(msg).encode("utf-8", "replace").decode("utf-8", "replace")[:300])))

['\u200c',
 'й',
 '>',
 '\xa0',
 'а',
 'т',
 'д',
 'e',
 'е',
 'ж',
 '\u2005',
 'n',
 ' ',
 'с',
 'w',
 'Д']

In [496]:
import re
reg = re.compile('[^a-zA-Zа-яА-Я0-9 ]')
print(reg.sub('', body))

new Дайджест  До 60 на пакетные предложения от SF Education SALE  Приветствую С вами на связи Роман Павлов ведущий вебинаров SF Education Делаем полезный перерыв в середине рабочего дня пройдите тест и узнайте как хорошо вы ориентируетесь в финансах А также приглашаю присоединиться к вебинарам этой недели Продуктивного дня и до встречи на вебинарах Анонсы SF Education Промо Никаких больше вечеринок Скидки на ВСЕкурсыдо 71 Подробнее С 11020 в SF Education повышаются цены на все курсы Последняя возможность получить востребованные навыки и знания выгодно Учитесь у признанных экспертов Вебинар 2309 в 2000 Лайфхаки в финансовом моделировании Регистрация Вебинар на основе курса Ивестиционный менеджер Подробнее о курсе Ярослав Малиновский  Инвестиционный аналитик АО Нацимбио ГК Ростех Вебинар 2409 в 2000 Доходная стратегия обзор рынков и инвестиционных решений Регистрация Вебинар на основе курса Ивестиционный менеджер Подробнее о курсе Дмитрий Абрамов  Директор Московского филиала БКС Брокер 